In [ ]:
#export
from .graph.edges import *
from .layer import Add, Concat
from math import ceil

In [1]:
from include.graph.edges import *
from include.layer import Add, Concat
from math import ceil

# Output verify helper function

In [80]:
#export
def _verify_output(gr, dest, out_shape, multi_input=False, layer=None):
    if dest == None:
        dest = gr.add_node(out_shape, multi_input, layer)
    elif gr.nodes[dest].shape != out_shape:
        raise Exception('Output shape does not match. got {}, expected {}'.format(out_shape, gr.nodes[dest].shape))
    elif multi_input and not gr.nodes[dest].multi_input:
        raise Exception('Output node is not a multi-input node.')
    
    return dest

# Basic layer functions

In [91]:
#export
def add_conv_layer(gr, src, dest=None, nf=None, ks=3, st=1, bias=True, identical=False):
    
    # Verify Input
    in_shape = gr.nodes[src].shape
    if nf == None:
        if dest == None:
            raise Exception('Output shape not defined.')
        else:
            nf = gr.nodes[dest].shape[0]
    ni = in_shape[0]
    
    # Get output shape
    edge = ConvEdge(src, 0, ni, nf, ks, st, bias=bias)
    if identical: edge.set_identical()
    out_shape = edge.calculate_output(in_shape)
    
    # Verify output and set
    dest = _verify_output(gr, dest, out_shape)
    edge.dest = dest
    
    gr.add_edge(edge)
    return dest

In [21]:
#export
def add_bn_layer(gr, src, dest=None, zero_bn=False):
    
    # Verify Input
    in_shape = gr.nodes[src].shape
    out_shape = in_shape
    nf = in_shape[0]
    
    # Get output shape
    edge = BatchNormEdge(src, 0, nf)
    if zero_bn: edge.set_zeros()
    out_shape = edge.calculate_output(in_shape)
    
    # Verify output and set
    dest = _verify_output(gr, dest, out_shape)
    edge.dest = dest
    
    gr.add_edge(edge)
    return dest

In [22]:
#export
def add_relu_layer(gr, src, dest=None):
    
    # Verify input
    in_shape = gr.nodes[src].shape
    
    # Get output shape
    edge = ReluEdge(src, 0)
    out_shape = edge.calculate_output(in_shape)
    
    # Verify output and set
    dest = _verify_output(gr, dest, out_shape)
    edge.dest = dest
    
    gr.add_edge(edge)
    return dest

In [23]:
#export
def add_linear_layer(gr, src, dest=None, no=None, bias=True):
    
    # Verify input
    in_shape = gr.nodes[src].shape
    ni = in_shape[0]
    
    if no == None:
        if dest == None:
            raise Exception('Output shape not defined')
        elif len(gr.nodes[dest].shape) != 1:
            raise Exception('Output dimension does not match.')
        else:
            no = gr.nodes[dest].shape[0]
            
    # Get output shape
    edge = LinearEdge(src, 0, ni, no, bias)
    out_shape = edge.calculate_output(in_shape)
    
    # Verify output and set
    dest = _verify_output(gr, dest, out_shape)
    edge.dest = dest
    
    gr.add_edge(edge)
    return dest

In [24]:
#export
def add_flatten_layer(gr, src, dest=None):
    
    # Verify input
    in_shape = gr.nodes[src].shape
    
    # Get output shape
    edge = FlattenEdge(src, 0)
    out_shape = edge.calculate_output(in_shape)
    
    # Verify output and set
    dest = _verify_output(gr, dest, out_shape)
    edge.dest = dest
    
    gr.add_edge(edge)
    return dest

In [25]:
#export
def add_pooling_layer(gr, src, dest=None, ks=2, method=None):
    
    # Verify input
    in_shape = gr.nodes[src].shape
    
    # Get output shape
    if method == 'avg':
        edge = AvgPoolingEdge(src, 0, kernel_size=ks)
    elif method == 'max':
        edge = MaxPoolingEdge(src, 0, kernel_size=ks)
    else:
        raise Exception('Pooling method undefined: {}'.format(method))
    out_shape = edge.calculate_output(in_shape)
    
    # Verify output and set
    dest = _verify_output(gr, dest, out_shape)
    edge.dest = dest
    
    gr.add_edge(edge)    
    return dest

In [26]:
#export
def add_adaptive_pooling_layer(gr, src, dest=None, target=None, method=None):
    
    # Verify input
    in_shape = gr.nodes[src].shape
    if isinstance(target, int): target = (target, target)
    elif isinstance(target, tuple): target = target
    else: raise Exception('Wrong target shape')
  
    out_shape = (in_shape[0],) + target

    # Verify output
    dest = _verify_output(gr, dest, out_shape)
    
    # Generate layer
    if method == 'avg':
        edge = AdaptiveAvgPoolingEdge(src, dest, output_size=target)
    elif method == 'max':
        edge = AdaptiveMaxPoolingEdge(src, dest, output_size=target)
    else:
        raise Exception('Pooling method undefined: {}'.format(method))
    
    gr.add_edge(edge)
    return dest

In [1]:
#export
def add_dropout_layer(gr, src, dest=None, p=0.2):
    
    # Verify input
    in_shape = gr.nodes[src].shape
    
    # Get output shape
    edge = DropoutEdge(src, 0, p=0.2)
    out_shape = edge.calculate_output(in_shape)
    
    # Verify output and set
    dest = _verify_output(gr, dest, out_shape)
    edge.dest = dest
    
    gr.add_edge(edge)
    return dest

In [82]:
#export
def add_id_layer(gr, src, dest=None):
    
    # Verify input
    in_shape = gr.nodes[src].shape
    if dest == None:
        dest = gr.add_node(in_shape)
    
    # Generate layer
    edge = IdenticalEdge(src, dest)
    
    gr.add_edge(edge)
    return dest

# Conv block helper functions

In [2]:
#export
def add_conv_block(gr, src, dest=None, nf=None, ks=3, st=1, p=0.2, zero_bn=False, act=True):
    
    # Generate conv layer and get output shape
    conv_node = add_conv_layer(gr, src, None, nf, ks, st, bias=False)
    do_node = add_dropout_layer(gr, conv_node, None, p=p)

    # Generate layer
    if act:
        bn_node = add_bn_layer(gr, do_node, None, zero_bn)
        dest = add_relu_layer(gr, bn_node, dest)
    else:
        dest = add_bn_layer(gr, do_node, dest, zero_bn)
    
    return dest

# ResNet helper functions

In [58]:
#export
def add_res_init_block(gr, src, dest=None):
    # Generate layer
    next = add_conv_block(gr, src, nf=64, ks=7, st=2, 
                          zero_bn=False, act=True)
    dest = add_pooling_layer(gr, next, dest, ks=2, method='max')

    return dest

In [31]:
#export
def add_res_end_block(gr, src, dest=None, no=None):
    
    # Verify input, generate output
    if no == None:
        if dest == None:
            raise Exception('Output shape not defined')
        elif len(gr.nodes[dest].shape) != 1:
            raise Exception('Output dimension does not match.')
        else:
            no = gr.nodes[dest].shape[0]
    
    # Generate layer
    next = add_adaptive_pooling_layer(gr, src, target=1, method='avg')
    next = add_flatten_layer(gr, next)
    dest = add_linear_layer(gr, next, dest)
    
    return dest

In [55]:
#export
def add_res_block(gr, src, dest=None, expansion=1, nh=None, st=1):
    
    # Verify input
    if nh == None:
        raise Excpeiton('Hidden layer shape not defined')
    
    in_shape = gr.nodes[src].shape
    ni = in_shape[0]
    nf = nh*expansion
    
    # Generate layer
    # branch 1
    if expansion == 1:
        next = add_conv_block(gr, src, nf=nh, ks=3, st=st)
        br1 = add_conv_block(gr, next, nf=nf, ks=3, zero_bn=True, act=False)
    else:
        next = add_conv_block(gr, src, nf=nh, ks=1)
        next = add_conv_block(gr, next, nf=nh, ks=3, st=st)
        br1 = add_conv_block(gr, next, nf=nf, ks=1, zero_bn=True, act=False)
    
    # branch 2
    next = add_pooling_layer(gr, src, ks=st, method='avg') if st != 1 else src
    br2 = add_conv_layer(gr, next, nf=nf, ks=1) if ni != nf else next    
    
    assert(gr.nodes[br1].shape == gr.nodes[br2].shape)
    out_shape = gr.nodes[br1].shape
    dest = _verify_output(gr, dest, out_shape, True, Add)
    
    add_id_layer(gr, br1, dest)
    add_id_layer(gr, br2, dest)
    
    return dest    

In [56]:
#export
def add_res_net(gr, src, dest=None, expansion=1, layers=[], c_out=None):
    
    # Verify input
    if c_out == None:
        if dest == None:
            raise Exception('Output shape not defined')
        elif len(gr.nodes[dest].shape) != 1:
            raise Exception('Output dimension does not match.')
        else:
            c_out = gr.nodes[dest].shape[0]

    in_shape = gr.nodes[src].shape
    out_shape = (c_out,)
    c_in = in_shape[0]
    
    # Verify output
    dest = _verify_output(gr, dest, out_shape)

    # Generate layer
    next = add_res_init_block(gr, src)

    nfs = [64//expansion, 64, 128, 256, 512]
    for id, depth in enumerate(layers):
        st = 1 if id==0 else 2
        for block in range(depth):
            next = add_res_block(gr, next, expansion=expansion, nh=nfs[id+1], st=st if block==0 else 1)
    
    dest = add_res_end_block(gr, next, dest)
    
    return dest

# DenseNet helper functions

In [68]:
#export
def add_dense_init_block(gr, src, dest=None, nf=64):
    # Generate layer
    next = add_conv_layer(gr, src, nf=64, ks=7, st=2, bias=False)
    dest = add_pooling_layer(gr, next, dest, ks=2, method='max')
    
    return dest

In [63]:
#export
def add_dense_layer(gr, src, dest=None,
                    growth_rate=32, bn_size=4, nf=64):
    
    next = add_bn_layer(gr, src)
    next = add_relu_layer(gr, next)
    next = add_conv_layer(gr, next, 
                          nf=bn_size*growth_rate, ks=1, st=1, bias=False)
    next = add_bn_layer(gr, next)
    next = add_relu_layer(gr, next)
    dest = add_conv_layer(gr, next, dest, 
                          nf=growth_rate, ks=3, st=1, bias=False)
    
    return dest

In [64]:
#export
def add_dense_block(gr, src, dest=None, 
                    growth_rate=32, bn_size=4, nf=64, num_layers=None):
    
    # Verify input
    if num_layers == None:
        raise Exception('Hidden layer shape not defined')
    
    in_shape = gr.nodes[src].shape
    ni = in_shape[0]
    out_shape = (ni+(num_layers)*growth_rate,) + in_shape[1:]
    
    # Verify output
    dest = _verify_output(gr, dest, out_shape, multi_input=True, layer=Concat)
    
    # Generate layers
    features = [src]
    next = src
    for i in range(1, num_layers + 1):
        if i == num_layers:
            cat = dest
        else:
            next_shape = (ni+i*growth_rate,) + in_shape[1:]
            cat = gr.add_node(next_shape, multi_input=True, layer=Concat)
        
        # branch 1
        br1 = add_dense_layer(gr, next, 
                              growth_rate=growth_rate, bn_size=bn_size, nf=nf)
        add_id_layer(gr, br1, cat)
        
        # branch 2~
        for feature in features:
            add_id_layer(gr, feature, cat)
        
        # Append new feature from branch 1
        features.append(br1)
        
        # Move iterator
        next = cat

    dest = next
    return dest

In [65]:
#export
def add_dense_transition(gr, src, dest=None, nf=None):
    if nf == None: raise Exception('Hidden layer shape not defined')

    next = add_bn_layer(gr, src)
    next = add_relu_layer(gr, next)
    next = add_conv_layer(gr, next, nf=nf, ks=1, st=1, bias=False)
    dest = add_pooling_layer(gr, next, dest, ks=2, method='avg')
    
    return dest

In [66]:
#export
def add_dense_net(gr, src, dest=None, growth_rate=32, bn_size=4, nf=64,
                  layers=[6, 12, 24, 16], c_out=None):
    # Verify input
    if c_out == None:
        if dest == None:
            raise Exception('Output shape not defined')
        elif len(gr.nodes[dest].shape) != 1:
            raise Exception('Output dimension does not match.')
        else:
            c_out = gr.nodes[dest].shape[0]
    
    # Generate layer
    num_features = nf
    next = add_dense_init_block(gr, src, nf=num_features)
    
    for i, nl in enumerate(layers):
        next = add_dense_block(gr, next, nf=nf, 
                               bn_size=bn_size, growth_rate=growth_rate, num_layers=nl)
        num_features = num_features + nl * growth_rate
        if i != len(layers) - 1:
            num_features = num_features // 2
            next = add_dense_transition(gr, next, nf=num_features)
    
    next = add_bn_layer(gr, next)
    next = add_flatten_layer(gr, next)
    dest = add_linear_layer(gr, next, dest)
    
    return dest

# Export

In [4]:
!python nb2py.py graph_transformer.ipynb

Converted graph_transformer.ipynb to exp/nb_graph_transformer.py


In [72]:
a = (1, 2)
b = tuple(2* x for x in a); b

(2, 4)